In [ ]:

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm.auto import tqdm
import warnings
import os
import pickle
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from torch.utils.data import TensorDataset, DataLoader

warnings.filterwarnings('ignore')


In [ ]:
is_debug = True
# ============================================================================
# CONFIG
# ============================================================================

class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction")
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 2 if is_debug else 200
    NFILE = 2 if is_debug else 19
    PATIENCE = 30
    LEARNING_RATE = 1e-3
    
    WINDOW_SIZE = 12  # 与推理保持一致
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94  # 与推理保持一致
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)


In [ ]:

# ============================================================================
# FEATURE ENGINEERING MODULES
# ============================================================================

def height_to_feet(height_str):
    """身高转换工具函数"""
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0


def create_base_features(input_df):
    """
    基础特征工程：物理属性、运动学、角色等基础特征
    """
    df = input_df.copy()
    
    # 基础物理特征
    df['player_height_feet'] = df['player_height'].apply(height_to_feet)
    
    # 运动学特征
    dir_rad = np.deg2rad(df['dir'].fillna(0))
    delta_t = 0.1
    df['velocity_x'] = (df['s'] + 0.5 * df['a'] * delta_t) * np.sin(dir_rad)
    df['velocity_y'] = (df['s'] + 0.5 * df['a'] * delta_t) * np.cos(dir_rad)
    df['acceleration_x'] = df['a'] * np.sin(dir_rad)
    df['acceleration_y'] = df['a'] * np.cos(dir_rad)
    
    # 角色特征
    df['is_offense'] = (df['player_side'] == 'Offense').astype(int)
    df['is_defense'] = (df['player_side'] == 'Defense').astype(int)
    df['is_receiver'] = (df['player_role'] == 'Targeted Receiver').astype(int)
    df['is_coverage'] = (df['player_role'] == 'Defensive Coverage').astype(int)
    df['is_passer'] = (df['player_role'] == 'Passer').astype(int)
    
    # 物理量特征
    mass_kg = df['player_weight'].fillna(200.0) / 2.20462
    df['momentum_x'] = df['velocity_x'] * mass_kg
    df['momentum_y'] = df['velocity_y'] * mass_kg
    df['kinetic_energy'] = 0.5 * mass_kg * (df['s'] ** 2)
    
    # 球相关特征
    if 'ball_land_x' in df.columns:
        ball_dx = df['ball_land_x'] - df['x']
        ball_dy = df['ball_land_y'] - df['y']
        df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        df['ball_direction_x'] = ball_dx / (df['distance_to_ball'] + 1e-6)
        df['ball_direction_y'] = ball_dy / (df['distance_to_ball'] + 1e-6)
        df['closing_speed'] = (
            df['velocity_x'] * df['ball_direction_x'] +
            df['velocity_y'] * df['ball_direction_y']
        )
    
    return df


def create_lag_features(df, window_size=8):
    """
    时序特征：滞后特征、EMA、滑动窗口特征
    """
    df = df.copy()
    
    # 按分组排序
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # Lag Features
    for lag in [1, 2, 3, 4, 5]:
        df[f'x_lag{lag}'] = df.groupby(gcols)['x'].shift(lag)
        df[f'y_lag{lag}'] = df.groupby(gcols)['y'].shift(lag)
        df[f'velocity_x_lag{lag}'] = df.groupby(gcols)['velocity_x'].shift(lag)
        df[f'velocity_y_lag{lag}'] = df.groupby(gcols)['velocity_y'].shift(lag)
    
    # EMA Features
    df['velocity_x_ema'] = df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    df['velocity_y_ema'] = df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    df['speed_ema'] = df.groupby(gcols)['s'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    
    # Rolling Features
    df['velocity_x_roll'] = df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    df['velocity_y_roll'] = df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    
    return df


def add_advanced_features(df):
    """
    高级特征：衍生特征、交互特征、战术特征
    """
    print("Adding advanced features...")
    df = df.copy()
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']

    # 位置差分特征
    if 'distance_to_ball' in df.columns:
        df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
        df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
        df['time_to_intercept'] = (df['distance_to_ball'] / 
                                  (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10)

    # 朝向球的特征
    if 'ball_direction_x' in df.columns:
        df['velocity_alignment'] = (
            df['velocity_x'] * df['ball_direction_x'] +
            df['velocity_y'] * df['ball_direction_y']
        )
        df['velocity_perpendicular'] = (
            df['velocity_x'] * (-df['ball_direction_y']) +
            df['velocity_y'] * df['ball_direction_x']
        )
        if 'acceleration_x' in df.columns:
            df['accel_alignment'] = (
                df['acceleration_x'] * df['ball_direction_x'] +
                df['acceleration_y'] * df['ball_direction_y']
            )

    # 速度差分特征
    if 'velocity_x' in df.columns:
        df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
        df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
        df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
        df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
        df['direction_change'] = df['direction_change'].apply(
            lambda x: x if abs(x) < 180 else x - 360 * np.sign(x)
        )

    # 场地位置特征
    df['dist_from_left'] = df['y']
    df['dist_from_right'] = 53.3 - df['y']
    df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
    df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])

    # 角色特定特征
    if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
        df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
        df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
    if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
        df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']

    # 时间特征
    df['frames_elapsed'] = df.groupby(gcols).cumcount()
    df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
        lambda x: x / (x.max() + 1)
    )

    # 基础物理衍生特征
    df['speed_squared'] = df['s'] ** 2
    df['accel_magnitude'] = np.sqrt(df['acceleration_x'] ** 2 + df['acceleration_y'] ** 2)
    df['orientation_diff'] = np.abs(df['o'] - df['dir'])
    df['orientation_diff'] = np.minimum(df['orientation_diff'], 360 - df['orientation_diff'])

    if 'nearest_opp_dist' in df.columns:
        df['pressure'] = 1 / np.maximum(df['nearest_opp_dist'], 0.5)
        df['under_pressure'] = (df['nearest_opp_dist'] < 3).astype(int)
        df['pressure_x_speed'] = df['pressure'] * df['s']

    if 'mirror_wr_vx' in df.columns:
        s_safe = np.maximum(df['s'], 0.1)
        df['mirror_similarity'] = (
                df['velocity_x'] * df['mirror_wr_vx'] +
                df['velocity_y'] * df['mirror_wr_vy']
        ) / s_safe
        df['mirror_offset_dist'] = np.sqrt(
            df['mirror_offset_x'] ** 2 + df['mirror_offset_y'] ** 2
        )
        df['mirror_alignment'] = df['mirror_similarity'] * df['is_coverage']

    return df


def get_opponent_features(input_df):
    """
    对手交互特征：计算球员与对手之间的空间和运动关系
    """
    features = []
    
    for (gid, pid), group in tqdm(input_df.groupby(['game_id', 'play_id']),
                                desc="🏈 Opponents", leave=False):
        last = group.sort_values('frame_id').groupby('nfl_id').last()

        if len(last) < 2:
            continue

        positions = last[['x', 'y']].values
        sides = last['player_side'].values
        speeds = last['s'].values
        directions = last['dir'].values
        roles = last['player_role'].values

        receiver_mask = np.isin(roles, ['Targeted Receiver', 'Other Route Runner'])

        for i, (nid, side, role) in enumerate(zip(last.index, sides, roles)):
            opp_mask = sides != side

            feat = {
                'game_id': gid, 'play_id': pid, 'nfl_id': nid,
                'nearest_opp_dist': 50.0, 'closing_speed': 0.0,
                'num_nearby_opp_3': 0, 'num_nearby_opp_5': 0,
                'mirror_wr_vx': 0.0, 'mirror_wr_vy': 0.0,
                'mirror_offset_x': 0.0, 'mirror_offset_y': 0.0,
                'mirror_wr_dist': 50.0,
            }

            if not opp_mask.any():
                features.append(feat)
                continue

            opp_positions = positions[opp_mask]
            distances = np.sqrt(((positions[i] - opp_positions) ** 2).sum(axis=1))

            if len(distances) == 0:
                features.append(feat)
                continue

            nearest_idx = distances.argmin()
            feat['nearest_opp_dist'] = distances[nearest_idx]
            feat['num_nearby_opp_3'] = (distances < 3.0).sum()
            feat['num_nearby_opp_5'] = (distances < 5.0).sum()

            # 计算相对速度
            my_vx = speeds[i] * np.sin(np.deg2rad(directions[i]))
            my_vy = speeds[i] * np.cos(np.deg2rad(directions[i]))
            opp_speeds = speeds[opp_mask]
            opp_dirs = directions[opp_mask]
            opp_vx = opp_speeds[nearest_idx] * np.sin(np.deg2rad(opp_dirs[nearest_idx]))
            opp_vy = opp_speeds[nearest_idx] * np.cos(np.deg2rad(opp_dirs[nearest_idx]))

            rel_vx = my_vx - opp_vx
            rel_vy = my_vy - opp_vy
            to_me = positions[i] - opp_positions[nearest_idx]
            to_me_norm = to_me / (np.linalg.norm(to_me) + 0.1)
            feat['closing_speed'] = -(rel_vx * to_me_norm[0] + rel_vy * to_me_norm[1])

            # 防守球员与接球手的镜像关系
            if role == 'Defensive Coverage' and receiver_mask.any():
                rec_positions = positions[receiver_mask]
                rec_distances = np.sqrt(((positions[i] - rec_positions) ** 2).sum(axis=1))

                if len(rec_distances) > 0:
                    closest_rec_idx = rec_distances.argmin()
                    rec_indices = np.where(receiver_mask)[0]
                    actual_rec_idx = rec_indices[closest_rec_idx]

                    rec_vx = speeds[actual_rec_idx] * np.sin(np.deg2rad(directions[actual_rec_idx]))
                    rec_vy = speeds[actual_rec_idx] * np.cos(np.deg2rad(directions[actual_rec_idx]))

                    feat['mirror_wr_vx'] = rec_vx
                    feat['mirror_wr_vy'] = rec_vy
                    feat['mirror_wr_dist'] = rec_distances[closest_rec_idx]
                    feat['mirror_offset_x'] = positions[i][0] - rec_positions[closest_rec_idx][0]
                    feat['mirror_offset_y'] = positions[i][1] - rec_positions[closest_rec_idx][1]

            features.append(feat)

    return pd.DataFrame(features)


def get_feature_columns(df):
    """
    获取特征列列表（包含所有特征）
    """
    base_feature_cols = [
        'x', 'y', 's', 'a', 'o', 'dir', 'frame_id',
        'ball_land_x', 'ball_land_y',
        'player_height_feet', 'player_weight',
        'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
        'momentum_x', 'momentum_y', 'kinetic_energy',
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
        'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
        'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
        'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
        'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
        'x_lag4', 'y_lag4', 'velocity_x_lag4', 'velocity_y_lag4',
        'x_lag5', 'y_lag5', 'velocity_x_lag5', 'velocity_y_lag5',
        'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
        'velocity_x_roll', 'velocity_y_roll',

        'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
        'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
        'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
        'dist_from_sideline', 'dist_from_endzone',
        'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
        'frames_elapsed', 'normalized_time', 'velocity_x_delta', 'velocity_y_delta',

        # 添加对手交互特征
        'nearest_opp_dist', 'closing_speed', 'num_nearby_opp_3', 'num_nearby_opp_5',
        'mirror_wr_vx', 'mirror_wr_vy', 'mirror_offset_x', 'mirror_offset_y', 'mirror_wr_dist',
        'pressure', 'under_pressure', 'pressure_x_speed',
        'mirror_similarity', 'mirror_offset_dist', 'mirror_alignment'
    ]
    
    # 合并所有特征列，只返回数据集中存在的
    all_feature_cols = base_feature_cols
    return [c for c in all_feature_cols if c in df.columns]


# ============================================================================
# 方向统一工具函数 (从别人方案中整合)
# ============================================================================

YARDS_TO_METERS = 0.9144
FPS = 10.0 
FIELD_LENGTH, FIELD_WIDTH = 120.0, 53.3

def wrap_angle_deg(s):
    # map to (-180, 180]
    return ((s + 180.0) % 360.0) - 180.0

def unify_left_direction(df: pd.DataFrame) -> pd.DataFrame:
    """Mirror rightward plays so all samples are 'left' oriented (x,y, dir, o, ball_land)."""
    if 'play_direction' not in df.columns:
        return df
    df = df.copy()
    right = df['play_direction'].eq('right')
    # positions
    if 'x' in df.columns: df.loc[right, 'x'] = FIELD_LENGTH - df.loc[right, 'x']
    if 'y' in df.columns: df.loc[right, 'y'] = FIELD_WIDTH  - df.loc[right, 'y']
    # angles in degrees
    for col in ('dir','o'):
        if col in df.columns:
            df.loc[right, col] = (df.loc[right, col] + 180.0) % 360.0
    # ball landing
    if 'ball_land_x' in df.columns:
        df.loc[right, 'ball_land_x'] = FIELD_LENGTH - df.loc[right, 'ball_land_x']
    if 'ball_land_y' in df.columns:
        df.loc[right, 'ball_land_y'] = FIELD_WIDTH  - df.loc[right, 'ball_land_y']
    return df

def build_play_direction_map(df_in: pd.DataFrame) -> pd.Series:
    """
    Return a Series indexed by (game_id, play_id) with values 'left'/'right'.
    """
    s = (
        df_in[['game_id','play_id','play_direction']]
        .drop_duplicates()
        .set_index(['game_id','play_id'])['play_direction']
    )
    return s

def apply_direction_to_df(df: pd.DataFrame, dir_map: pd.Series) -> pd.DataFrame:
    """
    Attach play_direction (if missing) and then unify to 'left'.
    """
    if 'play_direction' not in df.columns:
        dir_df = dir_map.reset_index()  # -> columns: game_id, play_id, play_direction
        df = df.merge(dir_df, on=['game_id','play_id'], how='left', validate='many_to_one')
    return unify_left_direction(df)

# ============================================================================
# 修改后的 MAIN SEQUENCE PREPARATION FUNCTION
# ============================================================================

def prepare_sequences_fixed(input_df, output_df=None, test_template=None, is_training=True, window_size=Config.WINDOW_SIZE):
    """
    准备序列数据的主函数（包含方向统一和所有特征工程）
    """
    # ===== 1. 方向统一处理 =====
    print("应用方向统一...")
    dir_map = build_play_direction_map(input_df)
    
    # 统一输入数据方向
    input_df_u = apply_direction_to_df(input_df, dir_map)
    
    if is_training:
        # 统一输出数据方向
        out_u = apply_direction_to_df(output_df, dir_map)
        target_rows = out_u
        target_groups = out_u[['game_id','play_id','nfl_id']].drop_duplicates()
    else:
        # 确保测试模板有play_direction
        if 'play_direction' not in test_template.columns:
            dir_df = dir_map.reset_index()
            test_template = test_template.merge(dir_df, on=['game_id','play_id'], how='left', validate='many_to_one')
        target_rows = test_template
        target_groups = target_rows[['game_id','play_id','nfl_id']].drop_duplicates()
    
    # 验证方向统一是否成功
    assert target_rows[['game_id','play_id','play_direction']].isna().sum().sum() == 0, \
        "play_direction merge failed; check (game_id, play_id) coverage"
    print("方向统一完成:", target_rows['play_direction'].value_counts(dropna=False).to_dict())
    
    # ===== 2. 特征工程 =====
    print("开始特征工程...")
    input_df_u = create_base_features(input_df_u)
    input_df_u = create_lag_features(input_df_u, window_size)
    opponent_features = get_opponent_features(input_df_u)
    input_df_u = input_df_u.merge(opponent_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    input_df_u = add_advanced_features(input_df_u)
    
    # 获取特征列
    feature_cols = get_feature_columns(input_df_u)
    print(f"使用 {len(feature_cols)} 个特征" + (" (训练)" if is_training else ""))
    
    # 设置索引用于分组操作
    input_df_u.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    grouped = input_df_u.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
    if is_training:
        sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
        
        for _, row in tqdm(target_groups.iterrows(), total = len(target_groups)):
            key = (row['game_id'], row['play_id'], row['nfl_id'])
            try:
                group_df = grouped.get_group(key)
            except KeyError:
                continue
            
            # 创建输入窗口
            input_window = group_df.tail(window_size)
            if len(input_window) < window_size:
                continue  # 训练时跳过长度不足的序列
            
            # 填充缺失值
            input_window = input_window.fillna(group_df.mean(numeric_only=True))
            seq = input_window[feature_cols].values
            
            # 处理NaN值
            if np.isnan(seq).any():
                continue  # 训练时跳过有NaN的序列
            
            sequences.append(seq)
            
            # 获取对应的目标值（注意：目标值已经是统一方向后的）
            out_grp = target_rows[
                (target_rows['game_id'] == row['game_id']) &
                (target_rows['play_id'] == row['play_id']) &
                (target_rows['nfl_id'] == row['nfl_id'])
            ].sort_values('frame_id')
            
            last_x = input_window.iloc[-1]['x']
            last_y = input_window.iloc[-1]['y']
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            targets_dx.append(dx)
            targets_dy.append(dy)
            targets_frame_ids.append(out_grp['frame_id'].values)
            
            sequence_ids.append({
                'game_id': key[0],
                'play_id': key[1],
                'nfl_id': key[2],
                'frame_id': input_window.iloc[-1]['frame_id'],
                'play_direction': input_window.iloc[-1]['play_direction']  # 保存方向信息用于推理时还原
            })
        
        return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids
    
    else:
        # 推理模式
        sequences, sequence_ids = [], []
        
        for _, row in tqdm(target_groups.iterrows(), desc="处理推理序列"):
            key = (row['game_id'], row['play_id'], row['nfl_id'])
            try:
                group_df = grouped.get_group(key)
            except KeyError:
                continue
            
            # 创建输入窗口
            input_window = group_df.tail(window_size)
            if len(input_window) < window_size:
                pad_len = window_size - len(input_window)
                pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
                input_window = pd.concat([pad_df, input_window], ignore_index=True)
            
            # 填充缺失值
            input_window = input_window.fillna(group_df.mean(numeric_only=True))
            seq = input_window[feature_cols].values
            
            # 处理NaN值
            if np.isnan(seq).any():
                seq = np.nan_to_num(seq, nan=0.0)
            
            sequences.append(seq)
            sequence_ids.append({
                'game_id': key[0],
                'play_id': key[1],
                'nfl_id': key[2],
                'frame_id': input_window.iloc[-1]['frame_id'],
                'play_direction': input_window.iloc[-1]['play_direction'],  # 保存方向信息
                'last_x': input_window.iloc[-1]['x'],  # 保存最后位置用于推理时还原
                'last_y': input_window.iloc[-1]['y']
            })
        
        return sequences, sequence_ids

In [ ]:

# ============================================================================
# LOSS (与训练一致)
# ============================================================================

class TemporalHuber(nn.Module):
    def __init__(self, delta=0.5, time_decay=0.03):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
    
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        
        huber = torch.where(
            abs_err <= self.delta,
            0.5 * err * err,
            self.delta * (abs_err - 0.5 * self.delta)
        )
        
        if self.time_decay > 0:
            L = pred.size(1)
            t = torch.arange(L, device=pred.device).float()
            weight = torch.exp(-self.time_decay * t).view(1, L)
            huber = huber * weight
            mask = mask * weight
        
        return (huber * mask).sum() / (mask.sum() + 1e-8)

# ============================================================================
# MODEL (与训练一致)
# ============================================================================

class ImprovedSeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.horizon = horizon
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        self.head = nn.Sequential(
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, horizon)
        )
    
    def forward(self, x):
        h, _ = self.gru(x)
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        ctx = ctx.squeeze(1)
        out = self.head(ctx)
        out = torch.cumsum(out, dim=1)
        return out


In [ ]:

# ============================================================================
# TRAINING (核心训练逻辑)
# ============================================================================

def prepare_targets(batch_axis, max_h):
    tensors, masks = [], []
    for arr in batch_axis:
        L = len(arr)
        padded = np.pad(arr, (0, max_h - L), constant_values=0).astype(np.float32)
        mask = np.zeros(max_h, dtype=np.float32)
        mask[:L] = 1.0
        tensors.append(torch.tensor(padded))
        masks.append(torch.tensor(mask))
    return torch.stack(tensors), torch.stack(masks)

def train_model(X_train, y_train, X_val, y_val, input_dim, horizon, config):
    device = config.DEVICE
    model = ImprovedSeqModel(input_dim, horizon).to(device)
    
    criterion = TemporalHuber(delta=0.5, time_decay=0.03)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    
    # Batches
    train_batches = []
    for i in range(0, len(X_train), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_train))
        bx = torch.tensor(np.stack(X_train[i:end]).astype(np.float32))
        by, bm = prepare_targets([y_train[j] for j in range(i, end)], horizon)
        train_batches.append((bx, by, bm))
    
    val_batches = []
    for i in range(0, len(X_val), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_val))
        bx = torch.tensor(np.stack(X_val[i:end]).astype(np.float32))
        by, bm = prepare_targets([y_val[j] for j in range(i, end)], horizon)
        val_batches.append((bx, by, bm))
    
    best_loss, best_state, bad = float('inf'), None, 0
    
    for epoch in range(1, config.EPOCHS + 1):
        model.train()
        train_losses = []
        
        for bx, by, bm in train_batches:
            bx, by, bm = bx.to(device), by.to(device), bm.to(device)
            pred = model(bx)
            loss = criterion(pred, by, bm)
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()
        val_losses = []
        with torch.no_grad():
            for bx, by, bm in val_batches:
                bx, by, bm = bx.to(device), by.to(device), bm.to(device)
                pred = model(bx)
                loss = criterion(pred, by, bm)
                val_losses.append(loss.item())
        
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        scheduler.step(val_loss)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{config.EPOCHS} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
            print(f"  ✅ Best val loss improved to {best_loss:.4f}, saving model")
        else:
            bad += 1
            if bad >= config.PATIENCE:
                print(f"  ❌ Early stopping at epoch {epoch} (no improvement for {bad} epochs)")
                break
    
    if best_state:
        model.load_state_dict(best_state)
    
    return model, best_loss


In [ ]:

# ============================================================================
# MAIN TRAINING FLOW
# ============================================================================


print("="*80)
print("NFL Big Data Bowl 2026 - TRAINING PIPELINE")
print("="*80)

# ===== 1. 加载数据 =====
print("\n[1/5] 加载数据...")
train_input_files = [Config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, Config.NFILE)]  # 只用第1周数据
train_output_files = [Config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, Config.NFILE)]

train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])

test_input = pd.read_csv(Config.DATA_DIR / "test_input.csv")
test_template = pd.read_csv(Config.DATA_DIR / "test.csv")

print(f"训练数据: {len(train_input):,} 行, {len(train_output):,} 行输出")
print(f"测试模板: {len(test_input):,} 行")

# ===== 2. 特征工程 =====
print("\n[2/5] 特征工程...")
sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = prepare_sequences_fixed(
    train_input, train_output, is_training=True, window_size=Config.WINDOW_SIZE
)

sequences = np.array(sequences, dtype=object)
targets_dx = np.array(targets_dx, dtype=object)
targets_dy = np.array(targets_dy, dtype=object)

print(f"生成序列数量: {len(sequences):,}")


In [ ]:

# ===== 3. 训练模型 (5折交叉验证) =====
print("\n[3/5] 开始训练模型...")
groups = np.array([d['game_id'] for d in sequence_ids])
gkf = GroupKFold(n_splits=Config.N_FOLDS)

models_x, models_y, scalers = [], [], []

for fold, (tr_idx, va_idx) in enumerate(gkf.split(sequences, groups=groups), 1):
    print(f"\n{'='*40}")
    print(f"Fold {fold}/{Config.N_FOLDS}")
    print(f"{'='*40}")
    
    X_tr = sequences[tr_idx]
    X_va = sequences[va_idx]
    y_tr_dx = [targets_dx[i] for i in tr_idx]
    y_va_dx = [targets_dx[i] for i in va_idx]
    y_tr_dy = [targets_dy[i] for i in tr_idx]
    y_va_dy = [targets_dy[i] for i in va_idx]
    
    # 特征标准化 (只用训练集fit)
    scaler = StandardScaler()
    scaler.fit(np.vstack([s for s in X_tr]))
    
    # 标准化数据
    X_tr_scaled = np.stack([scaler.transform(s) for s in X_tr])
    X_va_scaled = np.stack([scaler.transform(s) for s in X_va])
    
    input_dim = X_tr[0].shape[1]  # 特征维度
    
    # ===== 训练 X 方向模型 =====
    print("\n🔵 训练 X 方向模型...")
    model_x, best_loss_x = train_model(
        X_tr_scaled, y_tr_dx, X_va_scaled, y_va_dx,
        input_dim, Config.MAX_FUTURE_HORIZON, Config
    )
    models_x.append(model_x)
    scalers.append(scaler)
    print(f"✅ X 方向最佳 Val Loss: {best_loss_x:.4f}")
    
    # ===== 训练 Y 方向模型 =====
    print("\n🔴 训练 Y 方向模型...")
    model_y, best_loss_y = train_model(
        X_tr_scaled, y_tr_dy, X_va_scaled, y_va_dy,
        input_dim, Config.MAX_FUTURE_HORIZON, Config
    )
    models_y.append(model_y)
    print(f"✅ Y 方向最佳 Val Loss: {best_loss_y:.4f}")

# ===== 4. 保存模型 =====
print("\n[4/5] 保存模型...")
MODEL_SAVE_DIR = Path("./adddirec/")
MODEL_SAVE_DIR.mkdir(exist_ok=True)

for fold in range(Config.N_FOLDS):
    # 保存模型X
    torch.save(models_x[fold].state_dict(), MODEL_SAVE_DIR / f"model_x_fold{fold+1}.pth")
    # 保存模型Y
    torch.save(models_y[fold].state_dict(), MODEL_SAVE_DIR / f"model_y_fold{fold+1}.pth")
    # 保存Scaler
    joblib.dump(scalers[fold], MODEL_SAVE_DIR / f"scaler_fold{fold+1}.pkl")
    
    print(f"已保存 Fold {fold+1} 模型到 {MODEL_SAVE_DIR}")

print(f"\n🎉 训练完成！模型已保存到 {MODEL_SAVE_DIR}")
print(f"📂 请将此文件夹压缩并上传为 Kaggle Dataset，命名为如: nfl-models")
print(f"🔗 推理时请将模型上传到 /kaggle/input/nfl-models/ 目录")
print(f"📝 推理notebook会自动从该目录加载模型进行预测")

